In [1]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

# Indexing

In [3]:
dates=pd.date_range('1/2/2021', periods=8)
df = pd.DataFrame(np.random.randn(8, 4), columns=list("ABCD"),
                 index=dates)
df

,A,B,C,D
2021-01-02,1.629301,-0.009352,-0.905488,0.165572
2021-01-03,0.244092,1.313733,-0.347823,1.123616
2021-01-04,2.190245,-0.021858,1.136657,0.275776
2021-01-05,-0.676463,1.244879,0.791442,-0.511738
2021-01-06,-0.405205,0.502825,-0.922520,-1.285305
2021-01-07,-1.373553,0.056632,-0.666912,0.813599
2021-01-08,0.203159,1.045986,2.250007,-0.773191
2021-01-09,-1.254659,0.956169,-1.418185,-2.475736


In [8]:
s = df['A']
s[dates[5]]

-1.3735534007078942

In [10]:
df[['B', 'A']] = df[['A', 'B']]
df

,A,B,C,D
2021-01-02,-0.009352,1.629301,-0.905488,0.165572
2021-01-03,1.313733,0.244092,-0.347823,1.123616
2021-01-04,-0.021858,2.190245,1.136657,0.275776
2021-01-05,1.244879,-0.676463,0.791442,-0.511738
2021-01-06,0.502825,-0.405205,-0.922520,-1.285305
2021-01-07,0.056632,-1.373553,-0.666912,0.813599
2021-01-08,1.045986,0.203159,2.250007,-0.773191
2021-01-09,0.956169,-1.254659,-1.418185,-2.475736


In [18]:
df.loc[:, ['B', 'A']]= df[['A', 'B']].to_numpy()
df

,A,B,C,D
2021-01-02,-0.009352,1.629301,-0.905488,0.165572
2021-01-03,1.313733,0.244092,-0.347823,1.123616
2021-01-04,-0.021858,2.190245,1.136657,0.275776
2021-01-05,1.244879,-0.676463,0.791442,-0.511738
2021-01-06,0.502825,-0.405205,-0.922520,-1.285305
2021-01-07,0.056632,-1.373553,-0.666912,0.813599
2021-01-08,1.045986,0.203159,2.250007,-0.773191
2021-01-09,0.956169,-1.254659,-1.418185,-2.475736


In [30]:
sa = pd.Series([1, 2, 3], index=list('abc'))
sa.b
s['2021-01-07']

0.05663161423750495

In [31]:
dfa = df.copy()
dfa.A

2021-01-02   -0.009352
2021-01-03    1.313733
2021-01-04   -0.021858
2021-01-05    1.244879
2021-01-06    0.502825
2021-01-07    0.056632
2021-01-08    1.045986
2021-01-09    0.956169
Freq: D, Name: A, dtype: float64

In [32]:
print(sa)
sa.a = 5
sa

a    1
b    2
c    3
dtype: int64


a    5
b    2
c    3
dtype: int64

In [37]:
dfa.A = list(range(len(dfa.index))) # # ok if A already exists
dfa

dfa.E = list(range(len(dfa.index))) ## Error - UserWarning: Pandas doesn't allow columns to be created via a new attribute name
dfa

# use below form to create a new ˓→column
dfa['E'] = list(range(len(dfa.index)))
dfa

,A,B,C,D,E
2021-01-02,0,1.629301,-0.905488,0.165572,0
2021-01-03,1,0.244092,-0.347823,1.123616,1
2021-01-04,2,2.190245,1.136657,0.275776,2
2021-01-05,3,-0.676463,0.791442,-0.511738,3
2021-01-06,4,-0.405205,-0.922520,-1.285305,4
2021-01-07,5,-1.373553,-0.666912,0.813599,5
2021-01-08,6,0.203159,2.250007,-0.773191,6
2021-01-09,7,-1.254659,-1.418185,-2.475736,7


In [41]:
x = pd.DataFrame({'x': [1, 2, 3], 'y': [3, 4, 5]})
x.iloc[1] = {'x':9, 'y':99}
x

,x,y
0,1,3
1,9,99
2,3,5


In [43]:
df = pd.DataFrame({'one': [1., 2., 3.]})
df.two = [4, 5, 6] ## Error - UserWarning: Pandas doesn't allow columns to be created via a new attribute name
df

<ipython-input-43-400d0cb73b64>:2: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.two = [4, 5, 6]


,one
0,1.0
1,2.0
2,3.0


In [49]:
s[:5]
s[::2]
s[::-1]

#Note that setting works as well:
s2 = s.copy()
s2[:5] = 0
s2

2021-01-02    0.000000
2021-01-03    0.000000
2021-01-04    0.000000
2021-01-05    0.000000
2021-01-06    0.000000
2021-01-07    0.056632
2021-01-08    1.045986
2021-01-09    0.956169
Freq: D, Name: A, dtype: float64

In [51]:
df[:3]
df[::-1]

,one
2,3.0
1,2.0
0,1.0


In [61]:
df1 = pd.DataFrame(np.random.randn(5, 4), columns=list('ABCD'), index=pd.date_range('20130101', periods=5))
#df1.loc[2:4] #TypeError: cannot do slice indexing on DatetimeIndex with these indexers [2] of type int
df1.iloc[2:4]
df1.loc[[True, False, True, False, False]]

,A,B,C,D
2013-01-01,-0.264241,-0.412795,0.002391,-1.200910
2013-01-03,-0.325368,-0.292521,0.554763,-0.287031


In [66]:
s1 = pd.Series(np.random.randn(6), index=list('abcdef'))
s1.loc['c':]
s1.loc['b']
s1.loc['c':] = 0
s1

a   -0.243911
b   -0.918299
c    0.000000
d    0.000000
e    0.000000
f    0.000000
dtype: float64

In [71]:
df1 = pd.DataFrame(np.random.randn(6, 4), index=list('abcdef'), columns=list('ABCD'))
df1.loc[['a', 'b', 'd'], :]
df1.loc['a']
df1.loc[:, df1.loc['a'] > 0]

,C
a,1.021648
b,-0.074328
c,0.903928
d,-1.053766
e,-0.091051
f,-2.184279


In [74]:
mask = pd.array([True, False, True, False, pd.NA, False], dtype="boolean")
mask
df1[mask]

,A,B,C,D
a,-0.072105,-0.904690,1.021648,-0.573483
c,-0.603085,1.747245,0.903928,0.345961


In [111]:
dft = pd.DataFrame(np.random.randn(1000000, 6), columns=list('ABCDEF'))
dft.head()

,A,B,C,D,E,F
0,0.739179,0.007291,0.251444,0.312501,0.420025,-0.555057
1,0.551888,-0.916458,1.267054,-0.155771,1.163977,1.009823
2,-0.408805,0.464986,1.036291,0.060136,1.056598,0.197868
3,0.723118,0.060825,1.855839,0.498021,0.726894,-1.530539
4,-1.489332,-0.151683,-0.547574,-0.769638,1.414344,-0.316152
